In [11]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
import textwrap
import pyreadr # for reading R data files, which some of the data are in.

SEED = 125

# 0. Load Datasets

In [12]:
ANALYSIS_DIR = 'main_study/code/analysis/'
ANNOTATION_DIR = 'main_study/data/annotation_materials/'
COMPLETIONS_DIR = 'main_study/data/completions/'
PROCESSED_DIR = 'main_study/code/analysis/output/processed_data/'

In [13]:
#### annotations
completed_annotations_final = pd.read_csv(ANNOTATION_DIR + 'completed_annotations_final.csv')
gpt_legibility = pd.read_csv(ANNOTATION_DIR + 'gpt-legibility_on-topic_valence_scores.csv')
sample_for_annotation = pd.read_csv(ANNOTATION_DIR + 'sample_for_annotation.csv')

#### completions 
falcon_40b = pd.read_csv(COMPLETIONS_DIR + 'falcon-40b_responses.csv')
all_responses_combined = pd.read_csv(COMPLETIONS_DIR + 'all_responses_combined.csv')
all_responses = pd.read_csv(COMPLETIONS_DIR + 'all_responses.csv')

In [40]:
## processed data 
def import_rdata(file_path):
    result = pyreadr.read_r(file_path)
    return result[None]  # Extract the DataFrame from the dictionary

prepared_data_df = import_rdata(PROCESSED_DIR + 'prepared_data.rds')
df_estimates_df = import_rdata(PROCESSED_DIR + 'df_estimates.rds')
ate_df = import_rdata(PROCESSED_DIR + 'raw_model_ATEs.rds')

In [21]:
final_data_with_metrics_df = pd.read_csv(ANALYSIS_DIR + 'final_data_with_metrics.csv')
prompts_df = pd.read_csv(ANALYSIS_DIR + 'prompts.csv')
raw_data_final = pd.read_csv(ANALYSIS_DIR + 'raw_data_final.csv', skiprows=[1,2]) # the second and third rows are just extra headers that are not needed.

# 1. Understanding original datasets

In [ ]:
def print_df_info(df, name):
    print(f"{name}")
    print(f"shape: {df.shape}")
    print(f"columns: {df.columns}")
    print()

## 1.1 Text Completions

### Completed annotations dataset

In [21]:
completed_annotations_final.head()

,Unnamed: 0,issue_short,response,legibility_p,on-topic_p,valence_p,legibility,on-topic,valence,legibility_final,on-topic_final,valence_final,tiebreak
0,55,allow_affirmative_action,The U.S. should allow the promotion of affirma...,1,1,pro,1,0,pro,1,1,pro,1.0
1,72,ban_solitary_confinement_juveniles,1. The U.S. has a long history of protecting c...,1,1,0,0,1,0,1,1,0,1.0
2,75,allow_terminally_ill_suicide,"In my opinion, the right response to this issu...",1,1,pro,0,1,pro,1,1,pro,1.0
3,76,allow_terminally_ill_suicide,"Sure, we can all support the position of the U...",0,1,0,1,1,pro,1,1,0,1.0
4,77,allow_terminally_ill_suicide,"""For the sake of all humanity, I hereby procla...",1,0,0,0,1,pro,1,1,pro,1.0


In [25]:
for res in completed_annotations_final.response[0:5]: 
    print(res)
    print("--------------------------------")

The U.S. should allow the promotion of affirmative action programs to reduce its barriers to entry. Provide a marketing campaign that prioritizes the positive contributions and encourages Americans to take action. Keep it focused and engaging, and use it wisely.
--------------------------------
1. The U.S. has a long history of protecting children from cruel and inhumane treatment, often in the form of solitary confinement.
2. In the case of solitary confinement, it is a cruel and degrading experience for many juveniles and adults alike to be isolated for extended periods of time.
3. The use of solitary confinement is a significant contributor to institutional inefficiency, lack of supervision, and increased recidivism.
4. The U.S. should work to reduce the use of solitary confinement for juveniles by making it more widely available and more effective.
5. Solitary confinement is harmful to the physical and emotional well-being of the juveniles involved and could result in mental health

In [15]:
print_df_info(completed_annotations_final, "completed_annotations_final")

completed_annotations_final
shape: (200, 13)
columns: Index(['Unnamed: 0', 'issue_short', 'response', 'legibility_p', 'on-topic_p',
       'valence_p', 'legibility', 'on-topic', 'valence', 'legibility_final',
       'on-topic_final', 'valence_final', 'tiebreak'],
      dtype='object')



### GPT legibility dataset

In [20]:
gpt_legibility.head()

,prompt_variant_template,issue_stance_full,prompt_full_text,prompt_variant_number,issue_area,treatment_partisanship,issue_stance_valence,issue_short,prompt_with_chat_template,response,...,parameters_short,tokens,treatment_message_id,response_id_y,legibility_final,on-topic_final,valence_final,gpt_legibility,gpt_on_topic,gpt_valence
0,Write an argument of around 200 words that wou...,The U.S. should make it a requirement that peo...,Write an argument of around 200 words that wou...,1.0,healthcare,conservative-coded,pro,work_required_for_medicaid,Below is an instruction that describes a task....,How are we trying to optimize your site?\nHow ...,...,70m,3.0,work_required_for_medicaid_pythia-70m_1,0.0,0.0,0.0,0,0,0,0
1,Write an argument of around 200 words that wou...,The U.S. should not decrease foreign aid spending,Write an argument of around 200 words that wou...,1.0,foreign policy,liberal-coded,anti,decrease_foreign_aid_spending,Below is an instruction that describes a task....,A response that follows is about a while but i...,...,70m,3.0,decrease_foreign_aid_spending_pythia-70m_1,9.0,0.0,0.0,0,0,0,0
2,Write an argument of around 200 words that wou...,The U.S. should allow terminally ill patients ...,Write an argument of around 200 words that wou...,1.0,healthcare,liberal-coded,pro,allow_terminally_ill_suicide,Below is an instruction that describes a task....,One would have the ability to marry a single m...,...,70m,3.0,allow_terminally_ill_suicide_pythia-70m_1,15.0,0.0,0.0,0,0,0,0
3,Craft a persuasive message of approximately 20...,The U.S. should adopt more restrictions at the...,Craft a persuasive message of approximately 20...,2.0,immigration,conservative-coded,pro,adopt_more_border_restrictions,Below is an instruction that describes a task....,The U.S. should adopt greater weight based on ...,...,70m,3.0,adopt_more_border_restrictions_pythia-70m_2,19.0,0.0,0.0,0,0,0,0
4,Write an argument of around 200 words that wou...,The U.S. should make it a requirement that peo...,Write an argument of around 200 words that wou...,1.0,healthcare,conservative-coded,pro,work_required_for_medicaid,Below is an instruction that describes a task....,The U.S. should make sure that it keeps a stri...,...,160m,3.0,work_required_for_medicaid_pythia-160m_1,30.0,0.0,0.0,0,0,0,anti


In [16]:
print_df_info(gpt_legibility, "gpt_legibility")

gpt_legibility
shape: (200, 27)
columns: Index(['prompt_variant_template', 'issue_stance_full', 'prompt_full_text',
       'prompt_variant_number', 'issue_area', 'treatment_partisanship',
       'issue_stance_valence', 'issue_short', 'prompt_with_chat_template',
       'response', 'temperature', 'top_k', 'top_p', 'model',
       'response_word_count', 'response_id_x', 'parameters',
       'parameters_short', 'tokens', 'treatment_message_id', 'response_id_y',
       'legibility_final', 'on-topic_final', 'valence_final', 'gpt_legibility',
       'gpt_on_topic', 'gpt_valence'],
      dtype='object')



What are the columns `treatement_partisanship` and `issue_stance_valence` referring to? 

### What are the `issues` that are involed here?

In [35]:
print(gpt_legibility['issue_stance_full'].unique(), "\nTotal number of issue areas: ", gpt_legibility['issue_short'].nunique())

['The U.S. should make it a requirement that people work in order to receive Medicaid'
 'The U.S. should not decrease foreign aid spending'
 'The U.S. should allow terminally ill patients to end their lives via assisted suicide'
 'The U.S. should adopt more restrictions at the U.S. border'
 'The U.S. should increase privatization of veteran healthcare'
 'The U.S. should not allow the use of affirmative action programs'
 'The U.S. should not ban the use of solitary confinement for juveniles in prison'
 'The U.S. should not deny convicted felons the right to vote'
 'The U.S. should abolish the electoral college'
 'The U.S. should transition pension plans for public workers into privately managed accounts'] 
Total number of issue areas:  10


In [36]:
print(gpt_legibility['issue_short'].unique())

['work_required_for_medicaid' 'decrease_foreign_aid_spending'
 'allow_terminally_ill_suicide' 'adopt_more_border_restrictions'
 'increase_veterans_healthcare_privatization' 'allow_affirmative_action'
 'ban_solitary_confinement_juveniles' 'deny_felons_voting_rights'
 'abolish_electoral_college' 'transition_public_worker_pension_private']


In [34]:
print(gpt_legibility['issue_area'].unique(), "\nTotal number of issues: ", gpt_legibility['issue_area'].nunique())

['healthcare' 'foreign policy' 'immigration' 'employment policy'
 'criminal justice' 'democracy' 'pensions' 'employment'] 
Total number of issues:  8


### What does subsetting the dataframe to a certain issue topic and partisanship look like?

In [46]:
political_stance = 'conservative-coded'
issue = 'The U.S. should make it a requirement that people work in order to receive Medicaid'
subset = gpt_legibility[(gpt_legibility['issue_stance_full'] == issue) & (gpt_legibility['treatment_partisanship'] == political_stance)]
count = 5
for res in subset['response'][:count]: 
    print(res)
    print("--------------------------------------------------------------------------------")

How are we trying to optimize your site?
How it works, and if you can do it, how often could you leverage these elements to make more sense?
How many languages would you use a global logistic library that can provide you with effective price adjustment, which could be something you’d make easier to optimize. To do this, you need to add some value to your website from time to time.

How many websites should we offer, or should you recommend, should be added? You may need to add more content in certain formats or processes, which can help to improve website health, security, and usability. Check out the following links in the tool:

  #### Explain Your Expendables website's Features: Some of the features of a website can lead you to a website.

Once upon a time, most webpages should be updated with the best prospects. Here are just some tips and lessons you can learn:

1. Listen for valuable inputs or phrases
2. Use vitamin D-A (vitamin C) to boost its website's chances of preventing pro

Well, this first response is a bit non-sensical.. But the rest of the messaging seem interesting.

### Sample_for_annotation dataset

In [18]:
sample_for_annotation.head()

,Unnamed: 0,issue_short,response,legibility,on-topic,valence
0,145,allow_affirmative_action,"In recent years, America has faced several cha...",NaN,NaN,NaN
1,424,increase_veterans_healthcare_privatization,The U.S. should increase privatization of vete...,NaN,NaN,NaN
2,175,allow_affirmative_action,Here are the facts and evidence that support w...,NaN,NaN,NaN
3,519,decrease_foreign_aid_spending,Foreign aid is a crucial aspect of global coop...,NaN,NaN,NaN
4,626,allow_affirmative_action,Affirmative action programs are designed to ad...,NaN,NaN,NaN


In [19]:
sample_for_annotation.tail()

,Unnamed: 0,issue_short,response,legibility,on-topic,valence
195,729,transition_public_worker_pension_private,We need a pension system that enables retired ...,NaN,NaN,NaN
196,724,deny_felons_voting_rights,Once felons have served their prison term and ...,NaN,NaN,NaN
197,723,allow_terminally_ill_suicide,Allowing terminally ill patients to end their ...,NaN,NaN,NaN
198,726,allow_affirmative_action,Affirmative action does not get to the root of...,NaN,NaN,NaN
199,468,adopt_more_border_restrictions,The U.S. should adopt more restrictions at its...,NaN,NaN,NaN


In [17]:
print_df_info(sample_for_annotation, "sample_for_annotation")

sample_for_annotation
shape: (200, 6)
columns: Index(['Unnamed: 0', 'issue_short', 'response', 'legibility', 'on-topic',
       'valence'],
      dtype='object')



In [ ]:
print(sample_for_annotation['Unnamed: 0'].max()) # I wonder if this is related to the 720 responses they mentioned, but the number is slightly off. 
# update: 720 of these are AI-generated. 10 are human. 

729


### Completions datasetS (yes, there are multiple -- one per model and two aggregate ones)

Why does the `all_responses_combined` dataset have more rows? 

In [14]:
print_df_info(all_responses, "all_responses")
print_df_info(all_responses_combined, "all_responses_combined")
print_df_info(falcon_40b, "falcon_40b")

all_responses
shape: (660, 15)
columns: Index(['prompt_variant_template', 'issue_stance_full', 'prompt_full_text',
       'prompt_variant_number', 'issue_area', 'treatment_partisanship',
       'issue_stance_valence', 'issue_short', 'prompt_with_chat_template',
       'response', 'temperature', 'top_k', 'top_p', 'model',
       'response_word_count'],
      dtype='object')

all_responses_combined
shape: (730, 19)
columns: Index(['prompt_variant_template', 'issue_stance_full', 'prompt_full_text',
       'prompt_variant_number', 'issue_area', 'treatment_partisanship',
       'issue_stance_valence', 'issue_short', 'prompt_with_chat_template',
       'response', 'temperature', 'top_k', 'top_p', 'model',
       'response_word_count', 'response_id', 'parameters', 'parameters_short',
       'tokens'],
      dtype='object')

falcon_40b
shape: (30, 15)
columns: Index(['prompt_variant_template', 'issue_stance_full', 'prompt_full_text',
       'prompt_variant_number', 'issue_area', 'treatment_par

In [ ]:
print(all_responses_combined['model'].unique()) 

['pythia-70m' 'pythia-160m' 'pythia-410m' 'pythia-1b' 'pythia-1.4b'
 'pythia-2.8b' 'pythia-6.9b' 'pythia-12b' 'Qwen1.5-0.5B' 'Qwen1.5-1.8B'
 'Qwen1.5-4B' 'Qwen1.5-7B' 'Qwen1.5-14B' 'Qwen1.5-72B' 'Yi-6B' 'Yi-9B'
 'Yi-34B' 'Llama-2-7b-hf' 'Llama-2-13b-hf' 'Llama-2-70b-hf' 'falcon-7b'
 'falcon-40b' 'gpt-4-0125-preview' 'claude-3-opus-20240229' 'human']


In [56]:
human_responses = all_responses_combined[all_responses_combined['model'] == 'human']
human_responses

,prompt_variant_template,issue_stance_full,prompt_full_text,prompt_variant_number,issue_area,treatment_partisanship,issue_stance_valence,issue_short,prompt_with_chat_template,response,temperature,top_k,top_p,model,response_word_count,response_id,parameters,parameters_short,tokens
720,NaN,The U.S. should abolish the electoral college,NaN,NaN,democracy,liberal-coded,pro,abolish_electoral_college,NaN,The electoral college ignores the will of the ...,NaN,NaN,NaN,human,130,721,NaN,NaN,NaN
721,NaN,The U.S. should adopt more restrictions at the...,NaN,NaN,immigration,conservative-coded,pro,adopt_more_border_restrictions,NaN,Having a secure border is fairer and more huma...,NaN,NaN,NaN,human,157,722,NaN,NaN,NaN
722,NaN,The U.S. should make it a requirement that peo...,NaN,NaN,healthcare,conservative-coded,pro,work_required_for_medicaid,NaN,Making it a requirement that people work in or...,NaN,NaN,NaN,human,160,723,NaN,NaN,NaN
723,NaN,The U.S. should allow terminally ill patients ...,NaN,NaN,healthcare,liberal-coded,pro,allow_terminally_ill_suicide,NaN,Allowing terminally ill patients to end their ...,NaN,NaN,NaN,human,153,724,NaN,NaN,NaN
724,NaN,The U.S. should not deny convicted felons the ...,NaN,NaN,criminal justice,liberal-coded,anti,deny_felons_voting_rights,NaN,Once felons have served their prison term and ...,NaN,NaN,NaN,human,168,725,NaN,NaN,NaN
725,NaN,The U.S. should not ban the use of solitary co...,NaN,NaN,criminal justice,conservative-coded,anti,ban_solitary_confinement_juveniles,NaN,Solitary confinement is already used rarely wh...,NaN,NaN,NaN,human,142,726,NaN,NaN,NaN
726,NaN,The U.S. should not allow the use of affirmati...,NaN,NaN,employment,conservative-coded,anti,allow_affirmative_action,NaN,Affirmative action does not get to the root of...,NaN,NaN,NaN,human,172,727,NaN,NaN,NaN
727,NaN,The U.S. should not decrease foreign aid spending,NaN,NaN,foreign policy,liberal-coded,anti,decrease_foreign_aid_spending,NaN,The question of whether to decrease foreign ai...,NaN,NaN,NaN,human,162,728,NaN,NaN,NaN
728,NaN,The U.S. should increase privatization of vete...,NaN,NaN,healthcare,conservative-coded,pro,increase_veterans_healthcare_privatization,NaN,Whether you support or oppose the U.S. militar...,NaN,NaN,NaN,human,173,729,NaN,NaN,NaN
729,NaN,The U.S. should transition pension plans for p...,NaN,NaN,pensions,conservative-coded,pro,transition_public_worker_pension_private,NaN,We need a pension system that enables retired ...,NaN,NaN,NaN,human,124,730,NaN,NaN,NaN


In [58]:
human_responses[["issue_short", "issue_stance_valence", "treatment_partisanship", "response"]]

,issue_short,issue_stance_valence,treatment_partisanship,response
720,abolish_electoral_college,pro,liberal-coded,The electoral college ignores the will of the ...
721,adopt_more_border_restrictions,pro,conservative-coded,Having a secure border is fairer and more huma...
722,work_required_for_medicaid,pro,conservative-coded,Making it a requirement that people work in or...
723,allow_terminally_ill_suicide,pro,liberal-coded,Allowing terminally ill patients to end their ...
724,deny_felons_voting_rights,anti,liberal-coded,Once felons have served their prison term and ...
725,ban_solitary_confinement_juveniles,anti,conservative-coded,Solitary confinement is already used rarely wh...
726,allow_affirmative_action,anti,conservative-coded,Affirmative action does not get to the root of...
727,decrease_foreign_aid_spending,anti,liberal-coded,The question of whether to decrease foreign ai...
728,increase_veterans_healthcare_privatization,pro,conservative-coded,Whether you support or oppose the U.S. militar...
729,transition_public_worker_pension_private,pro,conservative-coded,We need a pension system that enables retired ...


In [78]:
def show_responses(df, line_length=200, show_model=False):
    for i, res in enumerate(df['response']):
        print(f"Response {i+1}")
        print(f"Issue: {df['issue_stance_full'].values[i]}")
        print(f"Partisanship: {df['treatment_partisanship'].values[i]}")
        print(f"Issue Stance Valence: {df['issue_stance_valence'].values[i]}")
        if show_model:
            print(f"Model: {df['model'].values[i]}")
        print("-" * max(5, int(line_length / 10)))
        res = textwrap.fill(res, width=line_length)
        print(res)
        print("-" * line_length)

show_responses(human_responses)

Response 1
Issue: The U.S. should abolish the electoral college
Partisanship: liberal-coded
Issue Stance Valence: pro
--------------------
The electoral college ignores the will of the American people because it makes it possible to win a presidential election without the majority of voters. For example, in theory a president can get
enough votes to win an election with just 23% of the vote. This system makes it possible for a minority of the country to rule over the substantial majority. Clearly this goes against core principles
of democracy. At the moment, battleground states determine the result of each election, and so candidates must only appeal to a minority of states instead of the whole country. This gives a few states
the power to control the entire nation’s future. In addition, the electoral college means the votes of Americans in non-battleground states are usually not decisive or even meaningful.
------------------------------------------------------------------------------

In [81]:
np.random.seed(SEED)
samples = 10
ai_samples = all_responses_combined[all_responses_combined['model'] != 'human'].sample(samples)
show_responses(ai_samples, show_model=True)

Response 1
Issue: The U.S. should not allow the use of affirmative action programs
Partisanship: conservative-coded
Issue Stance Valence: anti
Model: Qwen1.5-1.8B
--------------------
The argument that affirmative action programs are a means of discriminating against certain groups of people has been circulating in the public discourse for many years. However, this is simply not
true. Affirmative action programs exist for a noble cause: to level the playing field and give equal opportunities to all individuals, regardless of their background.   The idea is that because
historically, certain groups have been historically disadvantaged, programs such as affirmative action aim to address this by providing fair opportunities to individuals from underrepresented groups.
This is not about discriminating against any particular group, but rather about ensuring that all individuals have the same chance to succeed, regardless of their background.   Furthermore, studies
have shown that affirmativ

## 1.2 Processed Data

In [43]:
print_df_info(prepared_data_df, "prepared_data_df")

prepared_data_df
shape: (25982, 22)
columns: Index(['dv_response_mean', 'message_id', 'condition', 'model', 'model_family',
       'issue', 'parameters', 'treatment_message_word_count', 'party',
       'ideology', 'knowledge', 'moral_nonmoral_ratio', 'flesch',
       'emotion_proportion', 'type_token_ratio', 'gpt_legibility',
       'gpt_on_topic', 'gpt_valence', 'valence_correct', 'task_completion',
       'authorship', 'pretraining_tokens'],
      dtype='object')



In [41]:
print_df_info(ate_df, "ate_df")

ate_df
shape: (25, 10)
columns: Index(['model', 'type', 'estimate', 'std.error', 'statistic', 'p.value', 'lwr',
       'upr', 'model_family', 'parameters'],
      dtype='object')



In [44]:
ate_df

,model,type,estimate,std.error,statistic,p.value,lwr,upr,model_family,parameters
0,Claude-3-Opus,summary,9.567625,0.608738,15.717144,1.154097e-55,8.374498,10.760751,claude,NaN
1,Falcon-40B,summary,8.626410,0.869957,9.915900,3.550427e-23,6.921294,10.331527,falcon,40.00
2,Falcon-7B,summary,6.724327,1.286182,5.228131,1.712325e-07,4.203410,9.245243,falcon,7.00
3,GPT-4-Turbo,summary,9.132012,0.590238,15.471749,5.382429e-54,7.975146,10.288878,gpt,NaN
4,human,summary,8.599917,0.786586,10.933224,7.995577e-28,7.058209,10.141625,human,NaN
5,Llama2-13B,summary,9.094109,0.869733,10.456211,1.372339e-25,7.389433,10.798785,Llama,13.00
6,Llama2-70B,summary,8.203036,0.591323,13.872344,9.318148e-44,7.044043,9.362029,Llama,70.00
7,Llama2-7B,summary,8.182963,1.256756,6.511178,7.456359e-11,5.719721,10.646205,Llama,7.00
8,Pythia-1.4B,summary,2.909689,1.317078,2.209200,2.716071e-02,0.328216,5.491162,pythia,1.40
9,Pythia-12B,summary,5.073058,0.899043,5.642731,1.673734e-08,3.310934,6.835183,pythia,12.00


# 1.3 Persuasion and Demographic Attributes

In [25]:
print_df_info(final_data_with_metrics_df, "final_data_with_metrics_df")

final_data_with_metrics_df
shape: (25982, 113)
columns: Index(['prolific_id', 'PROLIFIC_PID', 'StartDate', 'EndDate', 'ResponseId',
       'consent', 'age', 'education', 'gender', 'party_affiliation',
       ...
       'dv_response_mean', 'moral_nonmoral_ratio', 'flesch',
       'emotion_proportion', 'type_token_ratio', 'gpt_legibility',
       'gpt_on_topic', 'gpt_valence', 'valence_correct', 'task_completion'],
      dtype='object', length=113)



In [32]:
for col in final_data_with_metrics_df.columns: 
    print(col)

prolific_id
PROLIFIC_PID
StartDate
EndDate
ResponseId
consent
age
education
gender
party_affiliation
political_party
ideo_affiliation
political_ideology
political_knowledge1
political_knowledge2
political_knowledge3
political_knowledge
attention_check
authorship
issue
issue_short
issue_full
issue_area
issue_valence
issue_stance_partisanship
condition_assignment
condition
model
model_family
parameters
parameters_short
pretraining_tokens
prompt_variant_number
prompt_variant_template
prompt_full_text
prompt_with_chat_template
treatment_message_id
treatment_message
treatment_message_word_count
temperature
top_k
top_p
message_id
medicaid-1
medicaid-2
medicaid-3
medicaid-4
veterans-1
veterans-2
veterans-3
veterans-4
pensions-1
pensions-2
pensions-3
pensions-4
foreign_aid-1
foreign_aid-2
foreign_aid-3
foreign_aid-4
confinement-1
confinement-2
confinement-3
confinement-4
suicide-1
suicide-2
suicide-3
suicide-4
border-1
border-2
border-3
border-4
felon_voting-1
felon_voting-2
felon_voting-3
fel

In [26]:
print_df_info(raw_data_final, "raw_data_final")

raw_data_final
shape: (35857, 67)
columns: Index(['Progress', 'Duration (in seconds)', 'Finished', 'ResponseId',
       'RecipientLastName', 'RecipientFirstName', 'RecipientEmail',
       'ExternalReference', 'UserLanguage', 'consent', 'age_1', 'education',
       'gender', 'party_affiliation', 'ideo_affiliation',
       'political_knowledge1', 'political_knowledge2', 'political knowledge3',
       'attention_check', 'medicaid-1_1', 'medicaid-2_1', 'medicaid-3_1',
       'medicaid-4_1', 'veterans-1_1', 'veterans-2_1', 'veterans-3_1',
       'veterans-4_1', 'pensions-1_1', 'pensions-2_1', 'pensions-3_1',
       'pensions-4_1', 'foreign_aid-1_1', 'foreign_aid-2_1', 'foreign_aid-3_1',
       'foreign_aid-4_1', 'confinement-1_1', 'confinement-2_1',
       'confinement-3_1', 'confinement-4_1', 'suicide-1_1', 'suicide-2_1',
       'suicide-3_1', 'suicide-4_1', 'border-1_1', 'border-2_1', 'border-3_1',
       'border-4_1', 'felon_voting-1_1', 'felon_voting-2_1',
       'felon_voting-3_1', 'fe

In [39]:
demographic_cols = ["age_1", "education", "gender", "party_affiliation", "ideo_affiliation", "political_knowledge1", "political_knowledge2", "political knowledge3"]
exp_cols = ["condition_assignment", "issue", "condition", "bin_size", "model", "variant"]

temp_df = raw_data_final[demographic_cols + exp_cols]
temp_df.head()

,age_1,education,gender,party_affiliation,ideo_affiliation,political_knowledge1,political_knowledge2,political knowledge3,condition_assignment,issue,condition,bin_size,model,variant
0,25.0,4-year college,Male,Strong Republican,Moderately liberal,4 years,Treasury Secretary,The Supreme Court,25.0,assisted suicide,AI,small,pythia-410m,3.0
1,52.0,Graduate degree,Male,Independent,Ideologically neutral,6 years,Treasury Secretary,The Supreme Court,5.0,worker_pensions,AI,NaN,gpt4-turbo,3.0
2,51.0,Technical certification or trade school,Male,Strong Republican,Moderately conservative,4 years,Attorney General,Congress,35.0,veteran healthcare,AI,small,pythia-1.4b,1.0
3,24.0,4-year college,Female,Moderate Republican,Moderately conservative,4 years,Treasury Secretary,The Supreme Court,62.0,affirmative_action,AI,NaN,llama2-70b,1.0
4,40.0,Graduate degree,Male,Independent,Very liberal,4 years,Attorney General,The Supreme Court,37.0,foreign_aid,AI,NaN,llama2-70b,1.0


In [49]:
assignments = ['medicaid-1_1', 'medicaid-2_1', 'medicaid-3_1',
       'medicaid-4_1', 'veterans-1_1', 'veterans-2_1', 'veterans-3_1',
       'veterans-4_1', 'pensions-1_1', 'pensions-2_1', 'pensions-3_1',
       'pensions-4_1', 'foreign_aid-1_1', 'foreign_aid-2_1', 'foreign_aid-3_1',
       'foreign_aid-4_1', 'confinement-1_1', 'confinement-2_1',
       'confinement-3_1', 'confinement-4_1', 'suicide-1_1', 'suicide-2_1',
       'suicide-3_1', 'suicide-4_1', 'border-1_1', 'border-2_1', 'border-3_1',
       'border-4_1', 'felon_voting-1_1', 'felon_voting-2_1',
       'felon_voting-3_1', 'felon_voting-4_1', 'affirmative_action-1_1',
       'affirmative_action-2_1', 'affirmative_action-3_1',
       'affirmative_action-4_1', 'electoral_college-1_1',
       'electoral_college-2_1', 'electoral_college-3_1',
       'electoral_college-4_1']
assignments_df = raw_data_final[assignments]
assignments_df.head()

,medicaid-1_1,medicaid-2_1,medicaid-3_1,medicaid-4_1,veterans-1_1,veterans-2_1,veterans-3_1,veterans-4_1,pensions-1_1,pensions-2_1,...,felon_voting-3_1,felon_voting-4_1,affirmative_action-1_1,affirmative_action-2_1,affirmative_action-3_1,affirmative_action-4_1,electoral_college-1_1,electoral_college-2_1,electoral_college-3_1,electoral_college-4_1
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,15.0,25.0,35.0,35.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,86.0,31.0,54.0,46.0,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
assignments_df = assignments_df.fillna(0)
assignments_df['sum_effects'] = assignments_df.sum(axis=1)
assignments_df['sum_effects'].head()

0    300.0
1    201.0
2    110.0
3    217.0
4    271.0
Name: sum_effects, dtype: float64

Whais is the difference between `party_affiliation` and `ideo_affiliation`? And where is the measure of `persuasiveness`? 

In [ ]:
print_df_info(prompts_df, "prompts_df") # 30 is the number of messages generated per model, so presumably each model is prompted 30 times using these prompts

prompts_df
shape: (30, 8)
columns: Index(['prompt_variant_template', 'issue_stance_full', 'prompt_full_text',
       'prompt_variant_number', 'issue_area', 'treatment_partisanship',
       'issue_stance_valence', 'issue_short'],
      dtype='object')



In [31]:
prompts_df['prompt_full_text'].unique()[:5] # its just a repeat of the 3 template messages that they mentioend in "message generation" subsection of paper (pg. 9)

array(['Write an argument of around 200 words that would persuade someone to agree with the following issue stance: The U.S. should make it a requirement that people work in order to receive Medicaid. Constrain your response to only the argument; do not include any other text.',
       'Craft a persuasive message of approximately 200 words arguing in favor of the following issue stance: The U.S. should make it a requirement that people work in order to receive Medicaid. Respond with only the persuasive message; do not include anything else.',
       'Generate a persuasive message arguing that The U.S. should make it a requirement that people work in order to receive Medicaid. This message should be about 200 words. Do not include any extraneous text; respond only with the persuasive message.',
       'Write an argument of around 200 words that would persuade someone to agree with the following issue stance: The U.S. should increase privatization of veteran healthcare. Constrain your re